In [ ]:
import numpy as np
import cupy as cp
import cupyx.scipy.sparse as sp
import cupyx.scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
from scipy.constants import pi, speed_of_light, elementary_charge, electron_mass, hbar

me_SI = electron_mass
hbar_SI = hbar
e_SI = elementary_charge
c_SI = speed_of_light

meV = e_SI * 1e-3
nm = 1e-9
ps = 1e-12

c = c_SI * ps / nm           
hbar_meV_ps = hbar_SI / (meV * ps)  
me = me_SI * c_SI**2 / meV / c**2    

hbar = hbar_meV_ps
m = me
omega = 2 / hbar
vQD = 15

x_min = -75
x_max = 150
t_min = 0
t_max = 20

x0 = 0
x1 = 75
t0 = 0
t1 = 2

In [ ]:
Nx = 5000
Nt = 5000

x_values = cp.linspace(x_min, x_max, Nx)
t_values = cp.linspace(t_min, t_max, Nt)

dx = x_values[1] - x_values[0]
dt = t_values[1] - t_values[0]

In [ ]:
laplacian = (cp.eye(Nx, k=-1) - 2 * cp.eye(Nx) + cp.eye(Nx, k=1)) / dx ** 2
H_kinetic = - (hbar ** 2 / (2 * m)) * laplacian

In [ ]:
A = (m * omega / (cp.pi * hbar)) ** 0.25
alpha = (m * omega) / (2.0 * hbar)

psi = A * cp.exp(-alpha * (x_values ** 2))
# psi /= np.sqrt(np.sum(psi ** 2) * dx)

In [ ]:
def quantum_center(t_values):
    xqd_arr = cp.zeros_like(t_values)
    
    for i, t in enumerate(t_values):
        if t < t1:
            xqd_arr[i] = x0
        elif t < t1 + (x1 - x0) / vQD:
            xqd_arr[i] = x0 + vQD * (t - t1)
        else:
            xqd_arr[i] = x1

    return xqd_arr

In [ ]:
psi_real_analytical = cp.zeros((Nx, Nt), dtype=cp.float64)
psi_img_analytical = cp.zeros((Nx, Nt), dtype=cp.float64)

psi_real_analytical[:, 0] = cp.real(psi)
psi_img_analytical[:, 0] = cp.imag(psi)

I = cp.eye(Nx)
xqd_arr = quantum_center(t_values)

for t_i in range(1, Nt):
    if t_i % 100 == 0:
        print(f"Step {t_i} / {Nt}")
    V = 0.5 * m * omega ** 2 * (x_values - xqd_arr[t_i]) ** 2
    H = H_kinetic + cp.diag(V)
    A = I + 1j * dt / (2 * hbar) * H
    B = I - 1j * dt / (2 * hbar) * H
    psi = spla.spsolve(sp.csc_matrix(A), B @ psi)
    # psi /= np.sqrt(np.sum(psi ** 2) * dx)
    psi_real_analytical[:, t_i] = cp.real(psi)
    psi_img_analytical[:, t_i] = cp.imag(psi)

In [ ]:
psi_real_analytical = cp.asnumpy(psi_real_analytical)
psi_img_analytical = cp.asnumpy(psi_img_analytical)
x_values = cp.asnumpy(x_values)
t_values = cp.asnumpy(t_values)

In [ ]:
plt.figure(figsize=(8, 6))
psi_sq = psi_real_analytical ** 2 + psi_img_analytical ** 2

plt.pcolormesh(x_values, t_values, psi_sq.transpose(), shading="auto")
plt.colorbar(label='|psi(x,t)|^2')
plt.xlabel('x')
plt.ylabel('t')
plt.title('Analytical |psi|^2')
plt.show()

In [ ]:
from scipy.integrate import simpson
from matplotlib.ticker import FormatStrFormatter

norms = []

for i in range(10000):
  psi_sq = psi_real_analytical[:, i] ** 2 + psi_img_analytical[:, i] ** 2
  normalization = simpson(psi_sq, x=x_values)
  norms.append(normalization)

plt.figure(figsize=(8,5))
plt.plot(t_values, norms, '-', label='Normalization')
plt.axhline(1.0, color='red', linestyle='--', label='Expected = 1')
plt.xlabel('Time (ps)')
plt.ylabel('Normalization')
plt.title('Wavefunction Normalization Over Time')
plt.legend()
plt.grid(True)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.10f'))

plt.show()